### **ETL Pipeline – University Student Lifestyle Data**

### Build an ETL (Extract, Transform, Load) pipeline to combine student lifestyle data from two universities, one shared via Google Sheets and the other via PostgreSQL. You will perform data extraction, cleaning, transformation, and finally load the cleaned dataset into Google BigQuery.

## Assignment:
1. Install and Import Required Libraries
2. Connect to PostgreSQL (University A)
● Fetch all records from the table student_lifestyle_dataset using a SQL query.
● Load the data into a DataFrame called university_a.

3. Extract Data from Google Sheets and load the data into a DataFrame called
University_b.
4. Add a new column named university in both DataFrames to identify their source:
● Assign the value 'A' for records extracted from PostgreSQL.
● Assign the value 'B' for records extracted from Google Sheets.

5. Display the top rows of both DataFrames to verify successful extraction.
6. Standardise Columns. Perform the following:
● Convert all column names to lowercase.
● Remove unwanted spaces or special characters in column names.
● Ensure both DataFrames have identical column names.
● If any mismatch exists, rename the columns appropriately.

7. Combine Both Datasets.
8. Data Cleaning & Transformation. Perform the following:
● Identify and handle missing values appropriately (fill/drop).
● Check column data types and fix incorrect ones.
● Detect and remove duplicate rows.
9. Connect to BigQuery and Load Data into BigQuery.
10. Verification
● Go to your BigQuery console.
● Verify that the table has been created and contains all records.

In [4]:
pip install psycopg2-binary


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import pandas as pd              # Data manipulation with DataFrames
from sqlalchemy import create_engine, text  # DB connection and executing SQL
import datetime                  # Work with dates and times
from google.cloud import bigquery  # Interact with BigQuery
from google.oauth2 import service_account  # Authenticate with Google Cloud
import warnings
warnings.filterwarnings('ignore')  # Suppress warning messages


In [52]:
# Connection parameters
db_user = "postgres"         # your PostgreSQL username
db_password = "tahir"             # Please enter your PostgreSQL password
db_host = "localhost"        # host (use '127.0.0.1' if localhost doesn't work)
db_port = "5432"             # default PostgreSQL port
db_name = "university"     # your database name

# Create the connection string
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create engine
engine = create_engine(connection_string)

In [53]:

try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        for row in result:
            print("Connected to:", row[0])
except Exception as e:
    print("Connection failed:", e)


Connected to: PostgreSQL 17.2 on x86_64-windows, compiled by msvc-19.42.34435, 64-bit


In [54]:
query = text("SELECT * FROM student_lifestyle_dataset ;")
query


In [55]:
# Fetch data into pandas DataFrame
A = pd.read_sql(query, engine)
A.head()

,student_id,study_hours_per_day,extracurricular_hours_per_day,sleep_hours_per_day,social_hours_per_day,physical_activity_hours_per_day,gpa,stress_level
0,1,2.5,1.0,7.5,2.0,0.5,3.1,Moderate
1,2,3.0,0.5,6.0,1.5,1.0,3.5,Low
2,3,1.5,2.5,5.5,3.0,0.5,2.8,High
3,4,4.0,1.5,7.0,2.5,1.0,3.9,Low
4,5,2.0,1.0,6.5,2.0,0.8,3.2,Moderate


In [56]:
url = "https://docs.google.com/spreadsheets/d/1FoGXo56G8aV129yR4CnuVRwWit_pf9ZVADO2bmR-oMk/export?format=csv"
B = pd.read_csv(url)

In [57]:
B.head()


,Student_ID,Study_Hours_Per_Day,Extracurricular_Hours_Per_Day,Sleep_Hours_Per_Day,Social_Hours_Per_Day,Physical_Activity_Hours_Per_Day,GPA,Stress_Level
0,1,6.9,3.8,8.7,2.8,1.8,2.99,Moderate
1,2,5.3,3.5,8.0,4.2,3.0,2.75,Low
2,3,5.1,3.9,9.2,1.2,4.6,2.67,Low
3,4,6.5,2.1,7.2,1.7,6.5,2.88,Moderate
4,5,8.1,0.6,6.5,2.2,6.6,3.51,High


In [64]:
A.head()

,student_id,study_hours_per_day,extracurricular_hours_per_day,sleep_hours_per_day,social_hours_per_day,physical_activity_hours_per_day,gpa,stress_level
0,1,2.5,1.0,7.5,2.0,0.5,3.1,Moderate
1,2,3.0,0.5,6.0,1.5,1.0,3.5,Low
2,3,1.5,2.5,5.5,3.0,0.5,2.8,High
3,4,4.0,1.5,7.0,2.5,1.0,3.9,Low
4,5,2.0,1.0,6.5,2.0,0.8,3.2,Moderate


In [69]:
A.shape

(10, 8)

In [73]:
# Step 6 (part): Convert all column names to lowercase

A.columns = A.columns.str.lower()
B.columns = B.columns.str.lower()


print("A columns:", list(A.columns))
print("B columns:", list(B.columns))


A columns: ['student_id', 'study_hours_per_day', 'extracurricular_hours_per_day', 'sleep_hours_per_day', 'social_hours_per_day', 'physical_activity_hours_per_day', 'gpa', 'stress_level']
B columns: ['student_id', 'study_hours_per_day', 'extracurricular_hours_per_day', 'sleep_hours_per_day', 'social_hours_per_day', 'physical_activity_hours_per_day', 'gpa', 'stress_level']


In [66]:
df= pd.concat([A,B], ignore_index=True)


In [67]:
df.head()

,student_id,study_hours_per_day,extracurricular_hours_per_day,sleep_hours_per_day,social_hours_per_day,physical_activity_hours_per_day,gpa,stress_level
0,1,2.5,1.0,7.5,2.0,0.5,3.1,Moderate
1,2,3.0,0.5,6.0,1.5,1.0,3.5,Low
2,3,1.5,2.5,5.5,3.0,0.5,2.8,High
3,4,4.0,1.5,7.0,2.5,1.0,3.9,Low
4,5,2.0,1.0,6.5,2.0,0.8,3.2,Moderate


In [71]:
B.shape

(2010, 8)

In [72]:
A.shape

(10, 8)

In [70]:
df.shape

(2020, 8)

In [74]:
df.isnull().sum()

student_id                         0
study_hours_per_day                0
extracurricular_hours_per_day      0
sleep_hours_per_day                0
social_hours_per_day               0
physical_activity_hours_per_day    0
gpa                                3
stress_level                       0
dtype: int64

In [75]:
df['gpa'].fillna(df['gpa'].mean(), inplace=True)


In [76]:
df.isnull().sum()

student_id                         0
study_hours_per_day                0
extracurricular_hours_per_day      0
sleep_hours_per_day                0
social_hours_per_day               0
physical_activity_hours_per_day    0
gpa                                0
stress_level                       0
dtype: int64

In [77]:
df.dtypes

student_id                           int64
study_hours_per_day                float64
extracurricular_hours_per_day      float64
sleep_hours_per_day                float64
social_hours_per_day               float64
physical_activity_hours_per_day    float64
gpa                                float64
stress_level                        object
dtype: object

In [84]:
duplicates = df.duplicated().sum()  # we not droping duplicates for now
duplicates

9

In [85]:
credentials = service_account.Credentials.from_service_account_file("datawarehouse8-550da3f60e12.json")
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

print("Connected to BigQuery project:", client.project)


Connected to BigQuery project: datawarehouse8


In [90]:
# Define destination table
table_id = f"{client.project}.student_lifestyle_dataset"

# Load data to BigQuery
job = client.load_table_from_dataframe(df, table_id)
job.result()  # wait until the job completes

print("Data successfully loaded to BigQuery:", table_id)

Data successfully loaded to BigQuery: datawarehouse8.student_lifestyle_dataset
